In [6]:
import csv

In [23]:
class CSVTable:
    def __init__(self,table_name,file_name, primary_key):
        self.table_name=table_name
        self.file_name= "../Data/" + file_name
        self.primary_key = primary_key
        self.reader = []
        self.writer = []
        self.header = []
        
    def load(self):
        csvfile = open(self.file_name)
        self.reader = csv.DictReader(csvfile)
        
    def load_header(self):
        self.load()
        self.header = list(list(self.reader)[0].keys())
        
    def save(self):
        with open(self.file_name, 'w', newline='') as csvfile:
            fieldnames = self.header
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in self.writer:
                writer.writerow(dict(row))
        
    def primary_key_error(self):
        for key in self.primary_key:
            if key not in self.header:
                return True
            else:
                return False
            
    def invalid_keys_error(self,t):
        for key, value in t.items():
            if key not in self.header:
                return True
            else:
                return False
            
    def insert_error(self,t):
        for key in self.primary_key:
            if not t.__contains__(key):
                return "err1"
        table = self.find_by_primary_key([value for key,value in t.items()])
        if table != []:
            return "err2"
            
    def find_by_primary_key(self, string, fields = None):
        if self.primary_key_error():
            print("Error: primary keys are not a subset of the columns in the underlying file/table")
        self.load()
        for row in self.reader:
            value_list = [row.get(key) for key in self.primary_key]             
            if value_list == string:
                if fields == None:
                    return row
                else:
                    return {key: value for key, value in row.items() if key in fields}
        return []
        
    def find_by_template(self, t, fields = None):
        if self.invalid_keys_error(t):
            return print("Error: find_by_template with invalid keys")
        self.load()
        dicts = self.reader
        for key,value in t.items():
            temp = []
            for row in dicts:
                if row.get(key) == value:
                    temp.append(row)
            dicts = temp            
        if fields == None:
            return dicts
        else:
            results = []
            for dic in dicts:
                results.append({key: value for key, value in dic.items() if key in fields})
            return results 
        
    def insert(self,t):
        if self.invalid_keys_error(t):
            return print("Error: insert with invalid keys")
        err = self.insert_error(t)
        if err == "err1":
            return print("Error: insert without primary keys")
        elif err == "err2":
            return print("Error: insert duplicate primary keys")
        self.load()
        self.writer = list(self.reader)
        self.writer.append(t)       
        
    def delete(self,t):
        if self.invalid_keys_error(t):
            return print("Error: delete with invalid keys")
        self.load()
        self.writer = []
        for row in self.reader:
            for key,value in t.items():
                if value != row[key]:
                    delete = False
                    break
                else:
                    delete = True
            
            if not delete:
                self.writer.append(row)


In [24]:
def test_template(test_name,names, file_name, primary_key, template, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template = ", template)
    print("Fields = ", fields)

    try:
        csvt = CSVTable(names, file_name, primary_key)
        csvt.load_header()
        
        r = csvt.find_by_template(template, fields)
        print("Result table:")
        print(r)
    except ValueError as ve:
        print("Exception = ", ve)

In [26]:
def test_primary_key(test_name,names, file_name, primary_key, values, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Values = ", values)
    print("Fields = ", fields)

    try:
        csvt = CSVTable(names, file_name, primary_key)
        csvt.load_header()

        r = csvt.find_by_primary_key(values, fields)
        print("Result table:")
        print(r)
    except ValueError as ve:
        print("Exception = ", ve)

In [37]:
def test_insert(test_name,names, file_name, primary_key, row, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)
    
    try:
        csvt = CSVTable(names, file_name, primary_key)
        csvt.load_header()
        
        csvt.insert(row)
        csvt.save()
    except ValueError as ve:
        print("Exception = ", ve)


In [41]:
def test_delete(test_name,names, file_name,primary_key, row, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)
    
    try:
        csvt = CSVTable(names, file_name, primary_key)
        csvt.load_header()
        
        csvt.delete(row)
        csvt.save()
    except ValueError as ve:
        print("Exception = ", ve)

In [27]:
test_template("Test1", "People", "People.csv", ["playerID"],
              {"birthMonth": "9", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"])
test_template("Test2", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
              {"playerID": "willite01", "yearID": "1961"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"])




*******************************
Test name =  Test1
Template =  {'birthMonth': '9', 'nameLast': 'Williams'}
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear', 'playerID']
Result table:
[{'birthMonth': '9', 'nameLast': 'Williams', 'playerID': 'willibe02', 'nameFirst': 'Bernie', 'birthYear': '1968'}, {'birthMonth': '9', 'nameLast': 'Williams', 'playerID': 'willido01', 'nameFirst': 'Don', 'birthYear': '1931'}, {'birthMonth': '9', 'nameLast': 'Williams', 'playerID': 'willido02', 'nameFirst': 'Don', 'birthYear': '1935'}, {'birthMonth': '9', 'nameLast': 'Williams', 'playerID': 'willini01', 'nameFirst': 'Nick', 'birthYear': '1993'}, {'birthMonth': '9', 'nameLast': 'Williams', 'playerID': 'willira01', 'nameFirst': 'Randy', 'birthYear': '1975'}, {'birthMonth': '9', 'nameLast': 'Williams', 'playerID': 'willist02', 'nameFirst': 'Stan', 'birthYear': '1936'}]


*******************************
Test name =  Test2
Template =  {'yearID': '1961', 'playerID': 'willite01'}
Fields =  ['playerI

In [29]:
test_primary_key("Test1", "People", "People.csv", ["playerID"],
              ["willibe02"], ["nameLast", "nameFirst", "birthMonth", "birthYear"])
   
test_primary_key("Test2", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
              ['willite01','BOS','1960','1'], ["playerID", "yearID", "teamID"])



*******************************
Test name =  Test1
Values =  ['willibe02']
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Result table:
{'birthMonth': '9', 'nameLast': 'Williams', 'nameFirst': 'Bernie', 'birthYear': '1968'}


*******************************
Test name =  Test2
Values =  ['willite01', 'BOS', '1960', '1']
Fields =  ['playerID', 'yearID', 'teamID']
Result table:
[]


In [35]:
test_insert("Insert Test 1", "People", "People.csv", ["playerID"],
            {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"},
            )

test_template("Find insert 1", "People", "People.csv", ["playerID"],
              {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"}, ["nameLast", "nameFirst", "playerID"]
              )

test_insert("Insert Test 2", "People", "People.csv", ["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
            )
test_template("Find Test 2", "People", "People.csv", ["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"}, ["nameLast", "nameFirst", "playerID"]
            )



*******************************
Test name =  Insert Test 1
Row to insert =  {'nameLast': 'Gao', 'playerID': 'zg2308', 'nameFirst': 'Zheyao'}


*******************************
Test name =  Find insert 1
Template =  {'nameLast': 'Gao', 'playerID': 'zg2308', 'nameFirst': 'Zheyao'}
Fields =  ['nameLast', 'nameFirst', 'playerID']
Result table:
[{'nameLast': 'Gao', 'playerID': 'zg2308', 'nameFirst': 'Zheyao'}]


*******************************
Test name =  Insert Test 2
Row to insert =  {'nameLast': 'Ferguson', 'playerID': 'dff1', 'nameFirst': 'Donald'}


*******************************
Test name =  Find Test 2
Template =  {'nameLast': 'Ferguson', 'playerID': 'dff1', 'nameFirst': 'Donald'}
Fields =  ['nameLast', 'nameFirst', 'playerID']
Result table:
[{'nameLast': 'Ferguson', 'playerID': 'dff1', 'nameFirst': 'Donald'}]


In [42]:
test_delete("Delete Test 1", "People", "People.csv", ["playerID"],
            {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"},
            )

test_template("Find Test 1", "People", "People.csv", ["playerID"],
              {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"}, ["nameLast", "nameFirst", "playerID"]
              )

test_delete("Delete Test 2", "People", "People.csv", ["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
            )
test_template("Find Test 2", "People", "People.csv", ["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"}, ["nameLast", "nameFirst", "playerID"]
            )



*******************************
Test name =  Delete Test 1
Row to insert =  {'nameLast': 'Gao', 'playerID': 'zg2308', 'nameFirst': 'Zheyao'}


*******************************
Test name =  Find Test 1
Template =  {'nameLast': 'Gao', 'playerID': 'zg2308', 'nameFirst': 'Zheyao'}
Fields =  ['nameLast', 'nameFirst', 'playerID']
Result table:
[]


*******************************
Test name =  Delete Test 2
Row to insert =  {'nameLast': 'Ferguson', 'playerID': 'dff1', 'nameFirst': 'Donald'}


*******************************
Test name =  Find Test 2
Template =  {'nameLast': 'Ferguson', 'playerID': 'dff1', 'nameFirst': 'Donald'}
Fields =  ['nameLast', 'nameFirst', 'playerID']
Result table:
[]
